In [116]:
import pandas as pd
import numpy as np
import pulp

## Part 1: Loading and preprocessing the datasets

In [161]:
orders = pd.read_excel("C:\\Users\\HOME\\Documents\\data_science\\ml_for_supply_chain_082024\\module4\\Supply chain logisitcs problem.xlsx", sheet_name='OrderList')
orders.dropna(axis = 1, how = 'all', inplace = True)
orders.dropna(axis = 0, how = 'all', inplace = True)
orders.head()

,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.30
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.94
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.20
3,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.16
4,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.34


In [162]:
orders.columns

Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')

In [163]:
freight_rates = pd.read_excel("C:\\Users\\HOME\\Documents\\data_science\\ml_for_supply_chain_082024\\module4\\Supply chain logisitcs problem.xlsx", sheet_name='FreightRates')
freight_rates.dropna(axis = 1, how = 'all', inplace = True)
freight_rates.dropna(axis = 0, how = 'all', inplace = True)
freight_rates.head()

,Carrier,orig_port_cd,dest_port_cd,minm_wgh_qty,max_wgh_qty,svc_cd,minimum cost,rate,mode_dsc,tpt_day_cnt,Carrier type
0,V444_6,PORT08,PORT09,250.0,499.99,DTD,43.2272,0.7132,AIR,2,V88888888_0
1,V444_6,PORT08,PORT09,65.0,69.99,DTD,43.2272,0.7512,AIR,2,V88888888_0
2,V444_6,PORT08,PORT09,60.0,64.99,DTD,43.2272,0.7892,AIR,2,V88888888_0
3,V444_6,PORT08,PORT09,50.0,54.99,DTD,43.2272,0.8272,AIR,2,V88888888_0
4,V444_6,PORT08,PORT09,35.0,39.99,DTD,43.2272,1.0552,AIR,2,V88888888_0


In [164]:
wh_capacities = pd.read_excel("C:\\Users\\HOME\\Documents\\data_science\\ml_for_supply_chain_082024\\module4\\Supply chain logisitcs problem.xlsx", sheet_name  = 'WhCapacities')
wh_capacities.dropna(axis = 1, how = 'all', inplace = True)
wh_capacities.dropna(axis = 0, how = 'all', inplace = True)
wh_capacities.head()

,Plant ID,Daily Capacity
0,PLANT15,11
1,PLANT17,8
2,PLANT18,111
3,PLANT05,385
4,PLANT02,138


In [165]:
products_per_plant = pd.read_excel("C:\\Users\\HOME\\Documents\\data_science\\ml_for_supply_chain_082024\\module4\\Supply chain logisitcs problem.xlsx", sheet_name = 'ProductsPerPlant')
products_per_plant.dropna(axis = 1, how = 'all', inplace = True)
products_per_plant.dropna(axis = 0, how = 'all', inplace = True)
products_per_plant.head()

,Plant Code,Product ID
0,PLANT15,1698815
1,PLANT17,1664419
2,PLANT17,1664426
3,PLANT17,1672826
4,PLANT17,1674916


In [166]:
wh_costs_per_unit = pd.read_excel("C:\\Users\\HOME\\Documents\\data_science\\ml_for_supply_chain_082024\\module4\\Supply chain logisitcs problem.xlsx", sheet_name = 'WhCosts')
wh_costs_per_unit.dropna(axis = 1, how = 'all', inplace = True)
wh_costs_per_unit.dropna(axis = 0, how = 'all', inplace = True)
wh_costs_per_unit.head()

,WH,Cost/unit
0,PLANT15,1.415063
1,PLANT17,0.428947
2,PLANT18,2.036254
3,PLANT05,0.488144
4,PLANT02,0.477504


In [167]:
# Переименование столбца WH в Plant ID
wh_costs_per_unit.rename(columns={'WH': 'Plant Code'}, inplace=True)

# Просмотр первых строк обновленного DataFrame
wh_costs_per_unit.head()

,Plant Code,Cost/unit
0,PLANT15,1.415063
1,PLANT17,0.428947
2,PLANT18,2.036254
3,PLANT05,0.488144
4,PLANT02,0.477504


In [168]:
# Объединение таблиц по ключу 'Plant Code' и добавление столбца 'Cost/unit'
products_per_plant = pd.merge(products_per_plant, wh_costs_per_unit[['Plant Code', 'Cost/unit']], on='Plant Code', how='left')

# Оставляем только необходимые столбцы
products_per_plant = products_per_plant[['Plant Code', 'Product ID', 'Cost/unit']]

# Просмотр первых строк обновленного DataFrame
products_per_plant.head()

,Plant Code,Product ID,Cost/unit
0,PLANT15,1698815,1.415063
1,PLANT17,1664419,0.428947
2,PLANT17,1664426,0.428947
3,PLANT17,1672826,0.428947
4,PLANT17,1674916,0.428947


In [169]:
ports = pd.read_excel("C:\\Users\\HOME\\Documents\\data_science\\ml_for_supply_chain_082024\\module4\\Supply chain logisitcs problem.xlsx", sheet_name  = 'PlantPorts')
ports.dropna(axis = 1, how = 'all', inplace = True)
ports.dropna(axis = 0, how = 'all', inplace = True)
ports.head()

,Plant Code,Port
0,PLANT01,PORT01
1,PLANT01,PORT02
2,PLANT02,PORT03
3,PLANT03,PORT04
4,PLANT04,PORT05


## Part 2: Building a dataset for optimization

We need to turn the shipping costs into a dictionary for easy lookup. We use the 'dict(zip(column1, column2))' paradigm.

In [170]:
shipping_costs = dict(zip(freight_rates['orig_port_cd'], freight_rates['rate']))

In [171]:
shipping_costs

{'PORT08': 0.1476,
 'PORT10': 0.11803052611157165,
 'PORT09': 13.2784,
 'PORT11': 0.20279999999999998,
 'PORT04': 0.1816,
 'PORT02': 0.124,
 'PORT03': 0.41159999999999997,
 'PORT07': 0.1972,
 'PORT05': 0.13520000000000001,
 'PORT06': 0.2756}

In [172]:
def get_plants(product_id):

    temp = products_per_plant[products_per_plant['Product ID'] == product_id]
    return tuple(np.unique(temp['Plant Code']))

In [173]:
''' 
a) Create a new column in the 'orders' dataframe called 'allowed_plants'. 
To do this, you'll need to apply the defined get_plants function using a lambda function.
'''

orders['allowed_plants'] = orders['Product ID'].apply(lambda x: get_plants(x))
orders

,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight,allowed_plants
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.300000,"(PLANT16,)"
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.940000,"(PLANT16,)"
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.200000,"(PLANT16,)"
3,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.160000,"(PLANT16,)"
4,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.340000,"(PLANT16,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9210,1.447305e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118,"(PLANT03,)"
9211,1.447319e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118,"(PLANT03,)"
9212,1.447322e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,245,0.294265,"(PLANT03,)"
9213,1.447145e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683430,PLANT03,PORT09,278,2.480000,"(PLANT03,)"


In [174]:
'''
b) Set the index of the 'orders' dataframe to be the 'Order ID'. Make sure you set the index in place. 
'''

orders.set_index('Order ID', inplace = True)

In [175]:
orders

,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight,allowed_plants
Order ID,,,,,,,,,,,,,,
1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.300000,"(PLANT16,)"
1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.940000,"(PLANT16,)"
1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.200000,"(PLANT16,)"
1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.160000,"(PLANT16,)"
1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.340000,"(PLANT16,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.447305e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118,"(PLANT03,)"
1.447319e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118,"(PLANT03,)"
1.447322e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,245,0.294265,"(PLANT03,)"


In [176]:
plant_ports = dict(zip(ports['Plant Code'], ports['Port']))

In [177]:
plant_ports

{'PLANT01': 'PORT02',
 'PLANT02': 'PORT03',
 'PLANT03': 'PORT04',
 'PLANT04': 'PORT05',
 'PLANT05': 'PORT06',
 'PLANT06': 'PORT06',
 'PLANT07': 'PORT02',
 'PLANT08': 'PORT04',
 'PLANT09': 'PORT04',
 'PLANT10': 'PORT02',
 'PLANT11': 'PORT04',
 'PLANT12': 'PORT04',
 'PLANT13': 'PORT04',
 'PLANT14': 'PORT07',
 'PLANT15': 'PORT08',
 'PLANT16': 'PORT09',
 'PLANT17': 'PORT10',
 'PLANT18': 'PORT11',
 'PLANT19': 'PORT04'}

### Writing the axiliary functions

In [178]:
def production_cost(order_id, plant):
    '''
    a) Return the production cost for a given order_id and plant (wahrehouse) name. 
    From the order id, you should first get the associated product id, which can be used to get the cost per unit.
    From here, multiply the cost per unit by the unit quantity to get the total production cost.
    '''
    
    t = orders.loc[order_id]
    prod_id = t['Product ID']
    pt = products_per_plant[products_per_plant['Product ID'] == prod_id]
    pt = pt[pt['Plant Code'] == plant]
    cpu = pt['Cost/unit']
    production_cost = cpu * t['Unit quantity']
    return production_cost.iloc[0]

In [179]:
def shipping_cost(order_id, plant):
    '''
    b) Return the shipping cost for a given order_id and plant (warehouse) name. 
    From the plant name, you should first get the associated port, which can be used to get the shipping cost per lb.
    From here, multiply the cost per lb by the weight to get the total shipping cost.
    '''
    
    t = orders.loc[order_id]
    w = t['Weight']
    port = plant_ports[plant]
    cp = shipping_costs[port]
    ship_cost = cp * w
    return ship_cost

In [180]:
def total_cost(order_id, plant):
    '''
    c) Return the total cost for a given order_id and plant (warehouse) name. 
    You should add the results of the two functions above. 
    '''

    return shipping_cost(order_id, plant) + production_cost(order_id, plant)

In [181]:
### We create a dictionary with the key-value pair 'orderId_plantName': total_cost.

order_costs = {}
for name, row in orders.iterrows():
    order_id = name
    for plant in row['allowed_plants']:   
        order_costs[str(order_id) + '_' + str(plant)] = total_cost(order_id, plant)

In [182]:
### We create a dictionary with the key-value pair 'plantName': list_of_orders.

plants = np.unique(ports['Plant Code'])

plant_orders = {}
for plant in plants:
    temp_list = []
    for name, row in orders.iterrows():
        if plant in row['allowed_plants']:  
            temp_list.append(str(name) + '_' + plant)
    plant_orders[plant] = temp_list

In [183]:
wh_capacities.columns

Index(['Plant ID', 'Daily Capacity '], dtype='object')

In [149]:
### We create a dictionary with the key-value pair 'plantName': capacity.

plant_cap = dict(zip(wh_capacities['Plant ID'], wh_capacities['Daily Capacity ']))

In [150]:
### We create a dictionary with the key-value pair 'orderID': orderID_plantName.

order_plants = {}
temp_dict = dict(zip(orders.index, orders['allowed_plants']))
for key in temp_dict:
    temp_list = []
    for pl in temp_dict[key]:
        temp_list.append(str(key) + '_' + pl)
    order_plants[key] = temp_list

In [151]:
build = pulp.LpVariable.dicts("Route",order_costs.keys(),0,None, pulp.LpInteger)
prob = pulp.LpProblem("Problem",pulp.LpMinimize)
prob += pulp.lpSum([build[b] * order_costs[b] for b in order_costs.keys()]), "Total Cost"

In [152]:
for plant in plant_orders:
    if len(plant_orders[plant]) > 0:
        prob += pulp.lpSum(build[p] for p in plant_orders[plant]) <= plant_cap[plant], "Total orders out of plant %s"%plant

In [153]:
for o in order_plants:
    prob += pulp.lpSum(build[p] for p in order_plants[o]) == 1, "Order_" + str(o) + "_filled"

## Part 3

In [154]:
'''
Part 3
'''

'\nPart 3\n'

In [155]:
''' 
a) Solve the linear programming problem and store its status in a variable called 'status'.
'''

status = pulp.LpStatus[prob.status]
for i in range(3):
    if status == 'Optimal':
        break
    prob.solve()

print("Status:", status)

Status: Not Solved


In [156]:
'''
b) Find the total cost to produce and ship all products and store the answer in a variable called 'total_cost'
Round the final answer to 2 decimal places (https://docs.python.org/3/library/functions.html#round).
'''
total_cost = round(pulp.value(prob.objective), 2)

print("Total Cost = ", str(total_cost))

Total Cost =  3077806.5
